<a href="https://colab.research.google.com/github/KoMurase/Practice-Hackathon/blob/master/Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import datasets, model_selection

features, labels = datasets.load_boston(return_X_y =True)
train_features, test_features, train_labels, test_labels = model_selection.train_test_split(features, labels, test_size=0.2)
train_features, valid_features, train_labels, valid_labels = model_selection.train_test_split(train_features, train_labels, test_size=0.1)


In [2]:
features

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [3]:
labels

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

In [4]:
print(train_features.shape)
print(train_labels.shape)
print(valid_features.shape)
print(valid_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(363, 13)
(363,)
(41, 13)
(41,)
(102, 13)
(102,)


In [0]:
import lightgbm as lgb

trains = lgb.Dataset(train_features, train_labels)
valids = lgb.Dataset(valid_features, valid_labels)
tests = lgb.Dataset(test_features, test_labels)

In [14]:
!pip install optuna

     |████████████████████████████████| 163kB 1.4MB/s 
     |████████████████████████████████| 1.1MB 4.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 112kB 7.3MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=ca09b805da5866c265fae564d92e2982878726b7bd616970af8e8d1297f7fee7
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.3.0-cp36-none-any.whl size=221121 sha256=a24fb30e22cecfa7c1117dde461d7fc4beaeb5db8386bfceb7fc525776d3f8b9
  Stored in directory: /root/.cache/pip/wheels/62/fd/

In [15]:
#Optunaで最適化
import optuna.integration.lightgbm as lgb

params = {
    'objective': 'mean_squared_error',
    'metric': 'rmse',
    'learning_rate':0.01

}

best_params, history = {}, []
model = lgb.train(params, trains, valid_sets=valids,
                    verbose_eval=False,
                    num_boost_round=100,
                    early_stopping_rounds=5,
                    best_params=best_params,
                    tuning_history=history)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

train is experimental (supported from v0.18.0). The interface can change in the future.

tune_num_leaves, val_score: 6.675619: 100%|##########| 20/20 [00:03<00:00,  5.90it/s]
tune_bagging_fraction_and_bagging_freq, val_score: 6.675619:   0%|          | 0/10 [00:00<?, ?it/s][I 2020-04-11 13:27:00,866] Finished trial#0 with value: 7.078992944285773 with parameters: {'bagging_fraction': 0.44115982727399095, 'bagging_freq': 4}. Best is trial#0 with value: 7.078992944285773.
tune_bagging_fraction_and_bagging_freq, val_score: 6.675619:  20%|##        | 2/10 [00:00<00:00,  9.22it/s][I 2020-04-11 13:27:01,022] Finished trial#1 with value: 6.994442665444247 with parameters: {'bagging_fraction': 0.48217747177737014, 'bagging_freq': 1}. Best is trial#1 with value: 6.994442665444247.
tune_bagging_fraction_and_bagging_freq, val_score: 6.668753:  30%|###       | 3/10 [00:00<00:00,  7.90it/s][I 2020-04-11 13:27:0

In [16]:
best_params # 'learning_rate':0.01

{'bagging_fraction': 0.9124048172220596,
 'bagging_freq': 1,
 'feature_fraction': 0.7799999999999999,
 'lambda_l1': 5.460482406088448e-08,
 'lambda_l2': 0.025561980810772284,
 'min_child_samples': 5,
 'num_leaves': 31}

In [17]:
#Optunaで最適化
import optuna.integration.lightgbm as lgb

params = {
    'objective': 'mean_squared_error',
    'metric': 'rmse',
    'learning_rate':0.1

}

best_params, history = {}, []
model = lgb.train(params, trains, valid_sets=valids,
                    verbose_eval=False,
                    num_boost_round=100,
                    early_stopping_rounds=5,
                    best_params=best_params,
                    tuning_history=history)

/usr/local/lib/python3.6/dist-packages/optuna/_experimental.py:90: ExperimentalWarning:

train is experimental (supported from v0.18.0). The interface can change in the future.

tune_num_leaves, val_score: 4.965875: 100%|##########| 20/20 [00:03<00:00,  5.93it/s]
tune_bagging_fraction_and_bagging_freq, val_score: 4.965875:   0%|          | 0/10 [00:00<?, ?it/s][I 2020-04-11 13:28:36,152] Finished trial#0 with value: 4.982079165291503 with parameters: {'bagging_fraction': 0.9584314682879786, 'bagging_freq': 7}. Best is trial#0 with value: 4.982079165291503.
tune_bagging_fraction_and_bagging_freq, val_score: 4.965875:  20%|##        | 2/10 [00:00<00:00, 10.29it/s][I 2020-04-11 13:28:36,285] Finished trial#1 with value: 5.611584391271463 with parameters: {'bagging_fraction': 0.6348800855759437, 'bagging_freq': 4}. Best is trial#0 with value: 4.982079165291503.
tune_bagging_fraction_and_bagging_freq, val_score: 4.851742:  30%|###       | 3/10 [00:00<00:00,  8.94it/s][I 2020-04-11 13:28:36,

In [18]:
best_params

{'bagging_fraction': 0.7332646511518158,
 'bagging_freq': 7,
 'feature_fraction': 0.7,
 'lambda_l1': 0.07954724855725478,
 'lambda_l2': 2.7264079431702472e-06,
 'min_child_samples': 10,
 'num_leaves': 31}

In [19]:
history

[{'action': 'tune_feature_fraction',
  'average_iteration_time': 0.0008906438237144833,
  'elapsed_secs': 0.0748140811920166,
  'trial': 0,
  'val_score': 5.187349960498245,
  'value': "{'feature_fraction': 0.4}"},
 {'action': 'tune_feature_fraction',
  'average_iteration_time': 0.0007887840270996094,
  'elapsed_secs': 0.07887840270996094,
  'trial': 1,
  'val_score': 5.056406031808018,
  'value': "{'feature_fraction': 0.5}"},
 {'action': 'tune_feature_fraction',
  'average_iteration_time': 0.0008261688053607941,
  'elapsed_secs': 0.05287480354309082,
  'trial': 2,
  'val_score': 5.244038181367036,
  'value': "{'feature_fraction': 0.6}"},
 {'action': 'tune_feature_fraction',
  'average_iteration_time': 0.0008343191707835478,
  'elapsed_secs': 0.07091712951660156,
  'trial': 3,
  'val_score': 4.96587543685954,
  'value': "{'feature_fraction': 0.7}"},
 {'action': 'tune_feature_fraction',
  'average_iteration_time': 0.0008288182710346423,
  'elapsed_secs': 0.04724264144897461,
  'trial': 